In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM


MODEL_ID = "/models/train/20241227-llama3.2-40M-finetune-1d9e/checkpoint-50000"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


In [3]:
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.quantization import QuantizationModifier


# Configure the simple PTQ quantization
recipe = QuantizationModifier(targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# Apply the quantization algorithm.
oneshot(model=model, recipe=recipe)

# Save the model.
SAVE_DIR = '/models/Llama-3.2-40M-fp8-dynamic'
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

2024-12-30T11:26:41.801502+0800 | main | WARNING - Process rank: 0, device: cuda:0, n_gpu: 8, distributed training: True, 16-bits training: False
2024-12-30T11:26:41.804243+0800 | main | INFO - Training/evaluation parameters TrainingArguments(
_n_gpu=8,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
clear_sparse_session=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspee

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: k-l-lambda (k-l-lambda-org). Use `wandb login --relogin` to force relogin


2024-12-30T11:26:44.900448+0800 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 0 modifiers


2024-12-30T11:26:55.156193+0800 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 0 modifiers
2024-12-30T11:26:55.157461+0800 | populate_datasets | INFO - Running oneshot without calibration data. This is expected for weight-only and dynamic quantization


2024-12-30T11:27:00.987969+0800 | one_shot | INFO - *** One Shot ***


...

2024-12-30T11:27:06.974586+0800 | initialize | INFO - Compression lifecycle initialized for 1 modifiers


manager stage: Modifiers finalized


2024-12-30T11:27:06.976520+0800 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2024-12-30T11:27:06.980823+0800 | get_model_compressor | INFO - Inferring a sparsity configuration requires a global sparsity calculation. This can be costly for large models. To skip the calculation of compression statistics set skip_compression_stats=True


Calculating model sparsity: 100%|██████████| 95/95 [00:00<00:00, 211.45it/s]
Calculating quantization compression ratio: 48it [00:00, 2347.78it/s]
Quantized Compression: 100%|██████████| 95/95 [00:00<00:00, 756.29it/s]


('/models/Llama-3.2-40M-fp8-dynamic/tokenizer_config.json',
 '/models/Llama-3.2-40M-fp8-dynamic/special_tokens_map.json',
 '/models/Llama-3.2-40M-fp8-dynamic/tokenizer.json')